#Functions

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!python /content/drive/MyDrive/child-level-data.py

                                            Abstract  ...    Y
0  (2 + 1)-dimensional non-linear optical waves t...  ...   12
1  (beta-amyloid (A beta) and tau pathology becom...  ...   74
2  (D)ecreasing of energy consumption and environ...  ...   68
3  (Hybrid) electric vehicles are assumed to play...  ...   26
4  (L)-3,4-Dihydroxyphenylalanine ((L)-DOPA) rema...  ...  115

[5 rows x 7 columns]
CS
Medical
Civil
ECE
biochemistry
MAE
Psychology


In [3]:
#SKLEARN LINRARIES
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,KFold

#NLTK LIBRARIES
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize,RegexpTokenizer
nltk.download('stopwords')
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
stop_words=stopwords.words("english")
from nltk.stem import PorterStemmer

from bs4 import BeautifulSoup
from IPython.display import clear_output
import numpy as np
import pandas as pd
import string
from matplotlib import pyplot as plt
from collections import Counter

#Tokenizing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

def Progress(string):
    global iterator
    global number_of_records
    percent=((iterator/number_of_records)*10)

    if percent%5==0:
        clear_output(wait=True)
        print(string+":"+str(percent)+"%")

    iterator=iterator+1

def remove_strip(text):
    return text.strip()

def remove_noise(text):


    soup=BeautifulSoup(text,"html.parser")
    text=soup.get_text()
    return text

def remove_punc(text):


    rem_punc=RegexpTokenizer(r'\w+')
    return ' '.join(rem_punc.tokenize(text))

def remove_stopwords(text):


    words=word_tokenize(text)
    return ' '.join([w.lower() for w in words if not w.lower() in stop_words])

def stemming(text):


    words=word_tokenize(text)
    porter=PorterStemmer()
    return ' '.join([porter.stem(w) for w in words ])



def Vocab(text):
  global freqdist
  words=word_tokenize(text)
  vocab.update(nltk.FreqDist(words).keys())

def build_vocab(documents):

  #maing vocab
  documents.apply(Vocab)

  return vocab

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
import numpy as np
import pandas as pd
import string
import os

areas=["CS","Civil","ECE","MAE","Medical","Psychology","biochemistry"]
child_data={}

for area in areas:
  #Make Directory For Child-Level
  if os.path.exists(f'/content/drive/MyDrive/{area}') == False:
    os.mkdir(f'/content/drive/MyDrive/{area}')


  child_data[area]=pd.read_csv(f"/content/{area}.csv")
  child_data[area].head() #show the head of the dataset
data=pd.read_csv(f'/content/drive/MyDrive/Data.csv')

In [5]:
child_data["CS"]

,index,Abstract,Domain
0,0,(2 + 1)-dimensional non-linear optical waves t...,Symbolic computation
1,5,(Objective) In order to increase classificatio...,Computer vision
2,21,[Purpose] This study verified that the smoothn...,Computer graphics
3,30,1. Camera trapping is a widely applied method ...,Operating systems
4,50,1. Species distribution modelling can be usefu...,Machine learning
...,...,...,...
6509,46961,Zero-difference balanced (ZDB) functions have ...,Cryptography
6510,46962,Zero-Suppressed Binary Decision Diagrams (ZDDs...,Data structures
6511,46969,Zinc is a modeling language in which a concept...,Data structures
6512,46981,Z-Wave is an implementation of home automation...,Data structures


#Preprocessing

In [ ]:
#before preprocessing
print("------------------------------------------------")
print(data["Abstract"][13])
print("------------------------------------------------")



#removing noise
iterator=1
data['Abstract']=data['Abstract'].apply(remove_noise)

#removeing puctuation
iterator=1
data['Abstract']=data['Abstract'].apply(remove_punc)

#removeing stopwords
iterator=1
data['Abstract']=data['Abstract'].apply(remove_stopwords)

#stemming
#     iterator=1
#     area_data['Abstract']=data['Abstract'].apply(stemming)


#after preprocessing
print("------------------------------------------------")
print(data["Abstract"][13])
print("------------------------------------------------")

#Tokenizer

In [ ]:
#number of distince frequency
print('--------------------------------------------------------------------------------------------------------')
vocab=Counter()
freqdist=len(build_vocab(data["Abstract"]))
print("Number Frequency "+str(freqdist))
print('--------------------------------------------------------------------------------------------------------')


# Tokenize our training data
print('--------------------------------------------------------------------------------------------------------')
print("Tokenizing...")
tokenizer = Tokenizer(freqdist-200)
tokenizer.fit_on_texts(data['Abstract'])
# Encode training data sentences into sequences
sequences = tokenizer.texts_to_sequences(data['Abstract'],)

# Get our training data word index
word_to_index = tokenizer.word_index

# Get max training sequence length
maxlen = max([len(x) for x in sequences])

#Pad the training sequences
sequences_padded = pad_sequences(sequences,maxlen=694)

# Output the results of our work
print("Word index:\n",len(word_to_index))
print("\nSequences:", sequences[0])
print("\nSequences Length:", len(sequences[0]))
print("\nPadded Sequences:", sequences_padded[0])
print("\nPadded Sequences Length:", len(sequences_padded[0]))
print('--------------------------------------------------------------------------------------------------------')

--------------------------------------------------------------------------------------------------------


NameError: name 'Counter' is not defined

In [ ]:
print(data["Abstract"][13])
print(word_to_index["purpose"])
print(sequences_padded[13])

#Model

In [ ]:
from sklearn.preprocessing import LabelBinarizer

parent_label=LabelBinarizer()
parent_label.fit_transform(data["Domain"])

child_labels={}
for area in areas:
  child_labels[area]=LabelBinarizer()
  child_labels[area].fit_transform(child_data[area]["Domain"])

#Prediction

In [ ]:
from keras.saving import load_model
parent_model=load_model("/content/drive/MyDrive/Data/parent_level_rnn_glove.100d.keras")
test_predict=[]
test_predict.append(sequences_padded[0])
test_predict=np.array(test_predict)
prediction = parent_model.predict(test_predict)
parent_label.classes_[np.argmax(prediction,axis=1)[0]]

In [ ]:
child_model=load_model(f"/content/drive/MyDrive/CS/child_level_rnn_glove.100d.keras")
test_predict=[]
test_predict.append(sequences_padded[0])
test_predict=np.array(test_predict)
prediction = child_model.predict(test_predict)
print(child_labels["CS"].classes_[np.argmax(prediction,axis=1)[0]])

In [ ]:
predicted_paths=[]



#parent-level predict
#-------------------------------------------------------------------------------
test_predict=[]
for index in range(0,len(sequences_padded)):
  test_predict.append(sequences_padded[index])
test_predict=np.array(test_predict)
parent_predictions=parent_model.predict(test_predict)
parent_predictions=np.argmax(parent_predictions,axis=1)
parent_predictions=[parent_label.classes_[predict].strip() for predict in parent_predictions]
#-------------------------------------------------------------------------------


#child-level predict
#-------------------------------------------------------------------------------
for area in areas:
  sequences_padded_test=[]
  indexs=[]
  child_model=load_model(f"/content/drive/MyDrive/{area}/child_level_rnn_glove.100d.keras")

  for index in range(0,len(parent_predictions)):
    if parent_predictions[index]==area:
      sequences_padded_test.append(sequences_padded[index])
      indexs.append(index)

  test_predict=[]
  for index in range(0,len(sequences_padded_test)):
    test_predict.append(sequences_padded_test[index])
  test_predict=np.array(test_predict)
  child_predictions=child_model.predict(test_predict)
  child_predictions=np.argmax(child_predictions,axis=1)
  child_predictions=[child_labels[area].classes_[predict].strip() for predict in child_predictions]

  for i in range(0,len(child_predictions)):
    predicted_paths.append([indexs[i],area,child_predictions[i]])
#-------------------------------------------------------------------------------

print(np.array(predicted_paths).shape)


 #Evaluate

In [ ]:
predicted_paths=pd.DataFrame(predicted_paths,columns=["index","Domain","area"])
predicted_paths=pd.DataFrame(predicted_paths.to_numpy(),index=predicted_paths["index"],columns=["index","Domain","area"]).sort_index()
predicted_paths=predicted_paths.drop(columns=["index"])
predicted_paths

In [ ]:
true_paths=data.loc[:,["Domain","area"]]
true_paths

In [ ]:
import array

tn=0

#TN
#------------------------------------------------------------------

true_set=[label.strip() for label in parent_label.classes_]
true_set.remove(true_paths["Domain"][79].strip())

predicted_set=[label.strip() for label in parent_label.classes_]
predicted_set.remove(predicted_paths["Domain"][79].strip())

intersect_set=list(set(true_set) & set(predicted_set))


tn=tn+len(intersect_set)


print("true_set",true_set)
print("predicted_set",predicted_set)
print("intersect_set",intersect_set)


true_set=[label.strip() for label in child_labels[true_paths["Domain"][79].strip()].classes_]
true_set.remove(true_paths["area"][79].strip())

predicted_set=[label.strip() for label in child_labels[predicted_paths["Domain"][79].strip()].classes_]
predicted_set.remove(predicted_paths["area"][79].strip())

intersect_set=list(set(true_set) & set(predicted_set))


tn=tn+len(intersect_set)


print("true_set",true_set)
print("predicted_set",predicted_set)
print("intersect_set:",intersect_set)
#------------------------------------------------------------------

In [ ]:
#FN
#------------------------------------------------------------------
fn=0

predicted_paths_list=[]
true_paths_list=[]
for index in range(0,len(predicted_paths["Domain"])):
  predicted_path=set([predicted_paths["Domain"][index],predicted_paths["area"][index]])
  true_path=set([true_paths["Domain"][index],true_paths["area"][index]])

  common_path=predicted_path &  true_path
  fn=fn+(len(true_path-common_path))
#------------------------------------------------------------------
print(fn)

In [ ]:
tp=0
tn=0
fp=0
fn=0
for index in range(0,len(true_paths)):
  #TN
  #------------------------------------------------------------------
  true_set=[label.strip() for label in parent_label.classes_]
  true_set.remove(true_paths["Domain"][index].strip())

  predicted_set=[label.strip() for label in parent_label.classes_]
  predicted_set.remove(predicted_paths["Domain"][index].strip())

  intersect_set=list(set(true_set) & set(predicted_set))
  tn=tn+len(intersect_set)



  true_set=[label.strip() for label in child_labels[true_paths["Domain"][index].strip()].classes_]
  true_set.remove(true_paths["area"][index].strip())

  predicted_set=[label.strip() for label in child_labels[predicted_paths["Domain"][index].strip()].classes_]
  predicted_set.remove(predicted_paths["area"][index].strip())

  intersect_set=list(set(true_set) & set(predicted_set))
  tn=tn+len(intersect_set)

  #------------------------------------------------------------------

  #------------------------------------------------------------------
  predicted_path=set([predicted_paths["Domain"][index].strip(),predicted_paths["area"][index].strip()])
  true_path=set([true_paths["Domain"][index].strip(),true_paths["area"][index].strip()])

  common_path=predicted_path &  true_path

  # print(predicted_path)
  # print(true_path)
  # print(common_path)

  tp=tp+len(common_path)
  # print(tp)
  fn=fn+(len(true_path-common_path))
  # print(fn)
  fp=fp+(len(predicted_path-common_path))
  # print(fp)
  #------------------------------------------------------------------




In [ ]:
accuracy=((tp+tn)/(tp+tn+fp+fn))*100
recall=(tp)/(tp+fn)*100
precision=(tp)/(tp+fp)*100


print("accuracy",accuracy)
print("recall",recall)
print("precision",precision)

#Evaluating Word Embedding

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from keras.saving import load_model
import joblib

parent_deep_learning_models=["cnn"]
child_deep_learning_models=["dnn"]
# word_embeddings=['glove.50d', 'glove.100d', 'glove.200d', 'glove.300d', 'GoogleNews-vectors', 'FastText']
word_embeddings=['GoogleNews-vectors']
Excel_Output=pd.DataFrame(columns=["first-level","second-level","word_embedding","tp","tn","fp","fn","accuracy","recall","precision","hP","hR"])

for parent_deep_learning_model in parent_deep_learning_models:
  for word_embedding in word_embeddings:

    #parent-level predict
    #-------------------------------------------------------------------------------
    if parent_deep_learning_model=="dnn":

      parent_model=load_model(f"/content/drive/MyDrive/Data/parent_level_dnn.keras")
      vocabulary = joblib.load("/content/drive/MyDrive/Data/vectorizer.pkl")
      cv=CountVectorizer(binary=False,ngram_range=(1,3),vocabulary=vocabulary)
      test_predict=cv.transform(data["Abstract"])

    else:

      parent_model=load_model(f"/content/drive/MyDrive/Data/parent_level_{parent_deep_learning_model}_{word_embedding}_1.keras")
      test_predict=[]
      for index in range(0,len(sequences_padded)):
        test_predict.append(sequences_padded[index])
      test_predict=np.array(test_predict)

    parent_predictions=parent_model.predict(test_predict)
    parent_predictions=np.argmax(parent_predictions,axis=1)
    parent_predictions=[parent_label.classes_[predict].strip() for predict in parent_predictions]
    #-------------------------------------------------------------------------------


    for child_deep_learning_model in child_deep_learning_models:
      predicted_paths=[]

      #child-level predict
      #-------------------------------------------------------------------------------
      for area in areas:

        if child_deep_learning_model=="dnn":
          data_cv=[]
          indexs=[]

          for index in range(0,len(parent_predictions)):
            if parent_predictions[index]==area:
              data_cv.append(data["Abstract"][index])
              indexs.append(index)

          child_model=load_model(f"/content/drive/MyDrive/{area}/child_level_dnn.keras")
          vocabulary = joblib.load(f"/content/drive/MyDrive/{area}/vectorizer.pkl")
          cv=CountVectorizer(binary=False,ngram_range=(1,3),vocabulary=vocabulary)
          test_predict=cv.transform(data_cv)

        else:
          sequences_padded_test=[]
          indexs=[]

          for index in range(0,len(parent_predictions)):
            if parent_predictions[index]==area:
              sequences_padded_test.append(sequences_padded[index])
              indexs.append(index)

          child_model=load_model(f"/content/drive/MyDrive/{area}/child_level_{child_deep_learning_model}_{word_embedding}.keras")

          test_predict=[]
          for index in range(0,len(sequences_padded_test)):
            test_predict.append(sequences_padded_test[index])
          test_predict=np.array(test_predict)


        child_predictions=child_model.predict(test_predict)
        child_predictions=np.argmax(child_predictions,axis=1)
        child_predictions=[child_labels[area].classes_[predict].strip() for predict in child_predictions]

        for i in range(0,len(child_predictions)):
          predicted_paths.append([indexs[i],area,child_predictions[i]])
      #-------------------------------------------------------------------------------

      print(np.array(predicted_paths).shape)


      #Evaluating The Hirearchical Classification
      #-------------------------------------------------------------------------------
      predicted_paths=pd.DataFrame(predicted_paths,columns=["index","Domain","area"])
      predicted_paths=pd.DataFrame(predicted_paths.to_numpy(),index=predicted_paths["index"],columns=["index","Domain","area"]).sort_index()
      predicted_paths=predicted_paths.drop(columns=["index"])

      true_paths=data.loc[:,["Domain","area"]]

      tp=0
      tn=0
      fp=0
      fn=0
      for index in range(0,len(true_paths)):
        #TN
        #------------------------------------------------------------------
        true_set=[label.strip() for label in parent_label.classes_]
        true_set.remove(true_paths["Domain"][index].strip())

        predicted_set=[label.strip() for label in parent_label.classes_]
        predicted_set.remove(predicted_paths["Domain"][index].strip())

        intersect_set=list(set(true_set) & set(predicted_set))
        tn=tn+len(intersect_set)



        true_set=[label.strip() for label in child_labels[true_paths["Domain"][index].strip()].classes_]
        true_set.remove(true_paths["area"][index].strip())

        predicted_set=[label.strip() for label in child_labels[predicted_paths["Domain"][index].strip()].classes_]
        predicted_set.remove(predicted_paths["area"][index].strip())

        intersect_set=list(set(true_set) & set(predicted_set))
        tn=tn+len(intersect_set)
        #--------------------------------------------------------------

        #------------------------------------------------------------------
        predicted_path=set([predicted_paths["Domain"][index].strip(),predicted_paths["area"][index].strip()])
        true_path=set([true_paths["Domain"][index].strip(),true_paths["area"][index].strip()])

        common_path=predicted_path &  true_path

        # print(predicted_path)
        # print(true_path)
        # print(common_path)

        tp=tp+len(common_path)
        # print(tp)
        fn=fn+(len(true_path-common_path))
        # print(fn)
        fp=fp+(len(predicted_path-common_path))
        # print(fp)
        #------------------------------------------------------------------
      accuracy=((tp+tn)/(tp+tn+fp+fn))*100
      recall=(tp)/(tp+fn)*100
      precision=(tp)/(tp+fp)*100

      hR=(tp/(46985*2))*100
      hP=(tp/(46985*2))*100

      print("#---------------------------")
      print(parent_deep_learning_model,child_deep_learning_model,word_embedding)
      print("accuracy",accuracy)
      print("recall",recall)
      print("precision",precision)

      print("hirearchical recall",(tp/(46985*2))*100)
      print("hirearchical precision",(tp/(46985*2))*100)

      print("#---------------------------")

      Excel_Output=pd.concat([Excel_Output,pd.DataFrame([{"first-level":parent_deep_learning_model,"second-level":child_deep_learning_model,"word_embedding":word_embedding,"tp":tp,"tn":tn,"fp":fp,"fn":fn,"accuracy":accuracy,"recall":recall,"precision":precision,"hP":hP,"hR":hR}])])
    #-------------------------------------------------------------------------------
Excel_Output.to_csv("/content/drive/MyDrive/Evaluate_OUTPU.csv")

 810/1469 [===============>..............] - ETA: 8s

KeyboardInterrupt: 

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# import joblib

# vocabulary = joblib.load("/content/drive/MyDrive/CS/vectorizer.pkl")
# print(vocabulary,"\n\n")
# print(data["Abstract"][21])

# cv=CountVectorizer(binary=False,ngram_range=(1,3),vocabulary=vocabulary)
# cv_predict=cv.transform([data["Abstract"][21]])

# from keras.saving import load_model
# child_model=load_model("/content/drive/MyDrive/CS/child_level_dnn.keras")

# prediction = child_model.predict(cv_predict)
# child_labels['CS'].classes_[np.argmax(prediction,axis=1)[0]]


#Evalute Document Embedding

In [ ]:
from gensim.models.doc2vec import Doc2Vec
from keras.saving import load_model


deep_learning_models=["dnn","rnn"]
document_embeddings=['doc2vec50', 'doc2vec100', 'doc2vec200', 'doc2vec300']


Excel_Output=pd.DataFrame(columns=["first-level","second-level","word_embedding","accuracy","tp","tn","fp","fn","recall","precision","hP","hR"])

for parent_deep_learning_model in deep_learning_models:
  for document_embedding in document_embeddings:

    #parent-level predict
    #-------------------------------------------------------------------------------
    if parent_deep_learning_model=="dnn":

      parent_model=load_model(f"/content/drive/MyDrive/Data/parent_level_dnn_doc_{document_embedding}.keras")
      test_predict=Doc2Vec.load(f"/content/drive/MyDrive/Doc2Vec/Data_{document_embedding}.model").dv.vectors

    else:

      parent_model=load_model(f"/content/drive/MyDrive/Data/parent_level_{parent_deep_learning_model}_doc_{document_embedding}.keras")
      test_predict=[]

      for index in range(0,len(data["Abstract"])):
        test_predict.append([index])
      test_predict=np.array(test_predict)

    parent_predictions=parent_model.predict(test_predict)
    parent_predictions=np.argmax(parent_predictions,axis=1)
    parent_predictions=[parent_label.classes_[predict].strip() for predict in parent_predictions]

    #-------------------------------------------------------------------------------



    for child_deep_learning_model in deep_learning_models:
      predicted_paths=[]

      #child-level predict
      #-------------------------------------------------------------------------------
      for area in areas:

        sequences_doc=Doc2Vec.load(f"/content/drive/MyDrive/Doc2Vec/Data_{document_embedding}.model").dv.vectors

        if child_deep_learning_model=="dnn":
          sequences_padded_test=[]
          indexs=[]

          for index in range(0,len(parent_predictions)):
            if parent_predictions[index]==area:
              sequences_padded_test.append(sequences_doc[index])
              indexs.append(index)

          child_model=load_model(f"/content/drive/MyDrive/{area}/child_level_dnn_doc_{document_embedding}.keras")


        else:
          sequences_padded_test=[]
          indexs=[]

          for index in range(0,len(parent_predictions)):
            if parent_predictions[index]==area:
              sequences_padded_test.append([index])
              indexs.append(index)

          child_model=load_model(f"/content/drive/MyDrive/{area}/child_level_{child_deep_learning_model}_doc_{document_embedding}.keras")



        test_predict=[]
        for index in range(0,len(sequences_padded_test)):
          test_predict.append(sequences_padded_test[index])
        test_predict=np.array(test_predict)

        child_predictions=child_model.predict(test_predict)
        child_predictions=np.argmax(child_predictions,axis=1)
        child_predictions=[child_labels[area].classes_[predict].strip() for predict in child_predictions]

        for i in range(0,len(child_predictions)):
          predicted_paths.append([indexs[i],area,child_predictions[i]])
      #-------------------------------------------------------------------------------


      print(np.array(predicted_paths).shape)


      #Evaluating The Hirearchical Classification
      #-------------------------------------------------------------------------------
      predicted_paths=pd.DataFrame(predicted_paths,columns=["index","Domain","area"])
      predicted_paths=pd.DataFrame(predicted_paths.to_numpy(),index=predicted_paths["index"],columns=["index","Domain","area"]).sort_index()
      predicted_paths=predicted_paths.drop(columns=["index"])

      true_paths=data.loc[:,["Domain","area"]]

      tp=0
      tn=0
      fp=0
      fn=0
      for index in range(0,len(true_paths)):
        #TN
        #------------------------------------------------------------------
        true_set=[label.strip() for label in parent_label.classes_]
        true_set.remove(true_paths["Domain"][index].strip())

        predicted_set=[label.strip() for label in parent_label.classes_]
        predicted_set.remove(predicted_paths["Domain"][index].strip())

        intersect_set=list(set(true_set) & set(predicted_set))
        tn=tn+len(intersect_set)



        true_set=[label.strip() for label in child_labels[true_paths["Domain"][index].strip()].classes_]
        true_set.remove(true_paths["area"][index].strip())

        predicted_set=[label.strip() for label in child_labels[predicted_paths["Domain"][index].strip()].classes_]
        predicted_set.remove(predicted_paths["area"][index].strip())

        intersect_set=list(set(true_set) & set(predicted_set))
        tn=tn+len(intersect_set)
        #--------------------------------------------------------------

        #------------------------------------------------------------------
        predicted_path=set([predicted_paths["Domain"][index].strip(),predicted_paths["area"][index].strip()])
        true_path=set([true_paths["Domain"][index].strip(),true_paths["area"][index].strip()])

        common_path=predicted_path &  true_path

        # print(predicted_path)
        # print(true_path)
        # print(common_path)

        tp=tp+len(common_path)
        # print(tp)
        fn=fn+(len(true_path-common_path))
        # print(fn)
        fp=fp+(len(predicted_path-common_path))
        # print(fp)
        #------------------------------------------------------------------
      accuracy=((tp+tn)/(tp+tn+fp+fn))*100
      recall=(tp)/(tp+fn)*100
      precision=(tp)/(tp+fp)*100

      hR=(tp/(46985*2))*100
      hP=(tp/(46985*2))*100

      print("#---------------------------")
      print(parent_deep_learning_model,child_deep_learning_model,document_embedding)
      print("accuracy",accuracy)
      print("recall",recall)
      print("precision",precision)

      print("hirearchical recall",(tp/(46985*2))*100)
      print("hirearchical precision",(tp/(46985*2))*100)

      print("#---------------------------")

      Excel_Output=pd.concat([Excel_Output,pd.DataFrame([{"first-level":parent_deep_learning_model,"second-level":child_deep_learning_model,"word_embedding":document_embedding,"tp":tp,"tn":tn,"fp":fp,"fn":fn,"accuracy":accuracy,"recall":recall,"precision":precision,"hP":hP,"hR":hR}])])
    #-------------------------------------------------------------------------------
Excel_Output.to_csv("/content/drive/MyDrive/Evaluate_DOC_OUTPU.csv")

181/181 [==============================] - 0s 2ms/step
(46985, 3)
#---------------------------
dnn dnn doc2vec50
accuracy 93.7722399887669
recall 55.86889432797702
precision 55.86889432797702
hirearchical recall 55.86889432797702
hirearchical precision 55.86889432797702
#---------------------------
181/181 [==============================] - 1s 2ms/step
(46985, 3)
#---------------------------
dnn rnn doc2vec50
accuracy 93.69239006287485
recall 55.30594870703416
precision 55.30594870703416
hirearchical recall 55.30594870703416
hirearchical precision 55.30594870703416
#---------------------------
185/185 [==============================] - 0s 2ms/step
(46985, 3)
#---------------------------
dnn dnn doc2vec100
accuracy 94.50697804638153
recall 60.12131531339789
precision 60.12131531339789
hirearchical recall 60.12131531339789
hirearchical precision 60.12131531339789
#---------------------------
185/185 [==============================] - 1s 3ms/step
(46985, 3)
#---------------------------
dn

#Disk Usage for Word Embeding

In [ ]:
import os


deep_learning_models=["rnn","cnn","dnn"]
word_embeddings=['glove.50d', 'glove.100d', 'glove.200d', 'glove.300d']
Excel_Output=pd.DataFrame(columns=["first-level","second-level","word_embedding","disk-usage"])
#parent-level-size
for parent_deep_learning_model in deep_learning_models:
  for word_embedding in word_embeddings:
    parent_file_size=0
    if parent_deep_learning_model=="dnn":
      parent_file_size=os.stat("/content/drive/MyDrive/Data/parent_level_dnn.keras").st_size

    else:
      parent_file_size=os.stat(f"/content/drive/MyDrive/Data/parent_level_{parent_deep_learning_model}_{word_embedding}.keras").st_size

    #child-level-size
    for child_deep_learning_model in deep_learning_models:
      child_file_size=0
      for area in areas:
        if child_deep_learning_model=="dnn":
          child_file_size=child_file_size+os.stat(f"/content/drive/MyDrive/{area}/child_level_dnn.keras").st_size
        else:
          child_file_size=child_file_size+os.stat(f"/content/drive/MyDrive/{area}/child_level_{child_deep_learning_model}_{word_embedding}.keras").st_size

      # print(f'File Size in MegaBytes is {file_size / (1024 * 1024 * 1024)}')
      file_size=(parent_file_size+child_file_size) / (1024 * 1024)
      Excel_Output=pd.concat([Excel_Output,pd.DataFrame([{"first-level":parent_deep_learning_model,"second-level":child_deep_learning_model,"word_embedding":word_embedding,"disk-usage":file_size}])])

Excel_Output.to_csv("/content/drive/MyDrive/Evaluate_OUTPU_Disk.csv")

#Disk Usage for Document Embeding

In [ ]:
import os


deep_learning_models=["dnn","rnn"]
document_embeddings=['doc2vec50', 'doc2vec100', 'doc2vec200', 'doc2vec300','Doc2Vecglove.50d','Doc2Vecglove.100d','Doc2Vecglove.200d','Doc2Vecglove.300d',"Doc2VecGoogleNews-vectors"]

Excel_Output=pd.DataFrame(columns=["first-level","second-level","document_embedding","disk-usage"])
#parent-level-size
for parent_deep_learning_model in deep_learning_models:
  for document_embedding in document_embeddings:
    parent_file_size=0
    if parent_deep_learning_model=="dnn":
      parent_file_size=os.stat(f"/content/drive/MyDrive/Data/parent_level_dnn_doc_{document_embedding}.keras").st_size

    else:
      parent_file_size=os.stat(f"/content/drive/MyDrive/Data/parent_level_{parent_deep_learning_model}_doc_{document_embedding}.keras").st_size

    #child-level-size
    for child_deep_learning_model in deep_learning_models:
      child_file_size=0
      for area in areas:
        if child_deep_learning_model=="dnn":
          child_file_size=child_file_size+os.stat(f"/content/drive/MyDrive/{area}/child_level_dnn_doc_{document_embedding}.keras").st_size
        else:
          child_file_size=child_file_size+os.stat(f"/content/drive/MyDrive/{area}/child_level_{child_deep_learning_model}_doc_{document_embedding}.keras").st_size

      # print(f'File Size in MegaBytes is {file_size / (1024 * 1024 * 1024)}')
      file_size=(parent_file_size+child_file_size) / (1024 * 1024)
      Excel_Output=pd.concat([Excel_Output,pd.DataFrame([{"first-level":parent_deep_learning_model,"second-level":child_deep_learning_model,"document_embedding":document_embedding,"disk-usage":file_size}])])

Excel_Output.to_csv("/content/drive/MyDrive/Evaluate_OUTPU_Doc_Disk.csv")

# Plot The Architecture

In [9]:
!pip install graphviz

In [10]:
!pip install pydot

In [28]:
!pip install keras==2.14
# !pip install keras

In [12]:
!pip install git+https://github.com/paulgavrikov/visualkeras --upgrade

  Cloning https://github.com/paulgavrikov/visualkeras to /tmp/pip-req-build-kgpoyq08
  Running command git clone --filter=blob:none --quiet https://github.com/paulgavrikov/visualkeras /tmp/pip-req-build-kgpoyq08
  Resolved https://github.com/paulgavrikov/visualkeras to commit ccc27c21c2275973ff94e7bd773c4f11f78916d7
  Preparing metadata (setup.py) ... done


In [13]:
!pip install keras-visualizer

In [12]:
from keras.saving import load_model
from keras.utils import plot_model

import visualkeras
from PIL import ImageFont

from keras_visualizer import visualizer


deep_learning_models=["dnn"]


parent_model=load_model(f"/content/drive/MyDrive/Data/parent_level_dnn.keras")

#CNN
# font = ImageFont.truetype("/content/Arial.ttf", 15)
# visualkeras.layered_view(parent_model,legend=True,to_file="parent_model.png",draw_volume=True,spacing=30,min_xy=40,max_xy=140,min_z=15 ,max_z=200,font=font) # without custom font
#DNN AND RNN
visualizer(parent_model,file_format="png",file_name="parent_model",view=True)

#Architecture
# plot_model(parent_model,to_file="parent_model.png",show_shapes=True)

# for child_deep_learning_model in deep_learning_models:
#   for area in areas:

#     #CNN
#     child_model=load_model(f"/content/drive/MyDrive/{area}/child_level_{child_deep_learning_model}_GoogleNews-vectors.keras")
#     font = ImageFont.truetype("/content/Arial.ttf", 15)
#     visualkeras.layered_view(child_model,legend=True,to_file=f"child_model_{area}.png",draw_volume=True,spacing=30,min_xy=40,max_xy=140,min_z=15 ,max_z=200,font=font)

    # visualizer(child_model,file_format="png",file_name=f"child_model_{area}",view=True)
    # plot_model(child_model,to_file=f"child_model_{area}.png",show_shapes=True)